Endereço Notebook:
https://colab.research.google.com/drive/1FxQvz-_JtK0TURKV4s4fHa9xskFKxjmF?usp=sharing

In [ ]:
!pip install -q kaggle
!rm -rf kaggle.json
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"danielmllerperes","key":"169691ef2966ba778c0d660f98414e92"}'}

In [ ]:
!rm -rf ~/.kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Download do Dataset do Kaggle a partir do token passado no json
!kaggle datasets download --force -d marlesson/news-of-the-site-folhauol

 98% 184M/187M [00:05<00:00, 43.9MB/s]
100% 187M/187M [00:05<00:00, 37.1MB/s]


# **Criar o DataFrame com os dados lidos diretamente da plataforma Kaggle**


In [ ]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

df = pd.read_csv("news-of-the-site-folhauol.zip")
#Ariui ao dataframe df o arquivo csv importado do Kaggle

# **Atualizar o SPACY e instalar os modelos pt_core_news_lg**

In [ ]:
#Importação do modelo Spacy e das Stop_words
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS

# **Instalar os datasets stopwords, punkt e rslp do nltk**

In [ ]:
#Importação dos pacotes para stop words do nltk e as demais bibliotecas
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

stop_words = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


# **Carregar os módulos usados ao longo desse notebook**

In [ ]:
#!pip install pyldavis &> /dev/null
#!pip install pyldavis==3.3.1

import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import numpy as np

#import pyLDAvis
#import pyLDAvis.sklearn

from wordcloud import WordCloud

import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain

from typing import List, Set, Any


SEED = 123

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **Filtrando os dados para utilizar apenas as notícias do ano de 2016 e da categoria "Mercado"**

Filtre os dados do DataFrame df e crie um DataFrame news_2016 que contenha apenas notícias de 2016 e da categoria mercado.

In [ ]:
#Criação do Dataframe com a categoria mercado e ano 2016
news_2016 = df[df['date'].str[:4] == '2016']
news_2016 = news_2016[news_2016['category'] == 'mercado']

#Remoção de conteudos descenessarios
del df
news_2016.drop(['category', 'subcategory', 'date', 'link'], axis=1)

# **NLTK Tokenizer and Stemmer**

Crie uma coluna no dataframe news_2016 contendo os tokens para cada um dos textos. Os tokens devem estar representados pelo radical das palavras (stem).Para tal, complete o conteúdo da função tokenize.

In [ ]:
#Importação dos pacotes para tokenização e stemming usando NLTK
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer
stemmer = RSLPStemmer()

In [ ]:
#Tokenização e stemming utilizando a biblioteca NLTK
def tokenize(text: str) -> List:
  tokens = word_tokenize(text)

  #Entra em cada item da lista tokenizada e aplica o stop_words
  list_tokens = [token for token in tokens if token.isalpha()]

  #Stemm de cada token
  stemmed_tokens = [stemmer.stem(token) for token in list_tokens]

  return stemmed_tokens

#Cria a coluna nltk_tokens contendo o resultado final de cada token
news_2016.loc[:, 'nltk_tokens'] = news_2016.text.progress_map(tokenize)

  0%|          | 0/7943 [00:00<?, ?it/s]

# **Criar uma documento SPACY para cada texto do dataset**

Crie uma coluna spacy_doc que contenha os objetos spacy para cada texto do dataset de interesse. Para tal, carregue os modelos pt_core_news_lg e aplique em todos os textos (pode demorar alguns minutos...)

In [ ]:
#Carga do modelo pt_core_news_lg da biblioteca spacy
!python -m spacy download pt_core_news_lg
nlp = spacy.load('pt_core_news_lg')

def spacy_nlp(a):
  return nlp(a)

news_2016.loc[:, 'spacy_doc'] = news_2016.text.progress_map(spacy_nlp)
news_2016.drop(['text'], axis=1)

# **Criar uma documento SPACY para cada texto do dataset**
Crie uma coluna spacy_doc que contenha os objetos spacy para cada texto do dataset de interesse. Para tal, carregue os modelos pt_core_news_lg e aplique em todos os textos (pode demorar alguns minutos...)


In [ ]:
#Criação de um documento Spacy aplicando os filtros aos tokens de remoção das stopwords, caracteres invalidos, nao alfabeticos e já lemmatizados.
def stopwords() -> Set:
    return set(list(nltk.corpus.stopwords.words('portuguese')) + list(STOP_WORDS))

complete_stopwords = stopwords()

def filter(w: spacy.lang.pt.Portuguese, StopWordsComplete) -> bool:
    if w not in StopWordsComplete and w not in ["o", "em", "em o", "em a", "ano"]:
      return True
    else: return False

def lemma(doc: spacy.lang.pt.Portuguese) -> List[str]:

    return [token.lemma_.lower() for token in doc if token.is_alpha and filter(token.lemma_, complete_stopwords)]

news_2016.loc[:, 'spacy_lemma'] = news_2016.spacy_doc.progress_map(lemma)

  0%|          | 0/7943 [00:00<?, ?it/s]

# Reconhecimento de entidades nomeadas
Crie uma coluna spacy_ner que armazene todas as organizações (APENAS organizações) que estão contidas no texto.

In [ ]:
#Criação de uma coluna para armazenar as entidades do tipo Organização
def NER(doc: spacy.lang.pt.Portuguese):

  entities = []
  for ent in doc:
    if ent.ent_type_ == 'ORG':
      entities.append(ent.text)
  
  return entities

news_2016.loc[:, 'spacy_ner'] = news_2016.spacy_doc.progress_map(NER)
news_2016.drop(['nltk_tokens'], axis=1)

## Bag-of-Words
Crie uma coluna tfidf no dataframe news_2016. Use a coluna spacy_lemma como base para cálculo do TFIDF. O número máximo de features que iremos considerar é 5000. E o token, tem que ter aparecido pelo menos 10 vezes (min_df) nos documentos.

In [ ]:
#Criação de uma coluna tfidf 
#Esta é a que melhor balanceia os tokens, pois considera a frequencia do termo no documento em analise e em todos os documentos da coluna, gerando uma classificação mais fidedigna

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

class Vectorizer:
  def __init__(self, doc_tokens: List):
      self.doc_tokens = doc_tokens
      self.tfidf = None
  
  def vectorizer(self):
    self.tfidf = TfidfVectorizer(lowercase=True, max_features=5000, min_df=10)
    doc_string = [' '.join(word) for word in self.doc_tokens]
    tfidf_matrix = self.tfidf.fit(doc_string)

  def __call__(self):
    if self.tfidf is None:
        self.vectorizer()
    return self.tfidf

doc_tokens = news_2016.spacy_lemma.values.tolist()
vectorizer = Vectorizer(doc_tokens)

def tokens2tfidf(tokens):
    array =  vectorizer().transform(tokens).toarray()[0]
    return array


news_2016.loc[:, 'tfidf'] = news_2016.spacy_lemma.progress_map(tokens2tfidf)

### Extração de Tópicos
Realize a extração de 9 tópicos usando a implementação do sklearn do algoritmo Latent Dirichlet Allocation. Como parâmetros, você irá usar o número máximo de iterações igual à 100 (pode demorar) e o random_seed igual a SEED que foi setado no início do notebook

In [ ]:
#Utilização do LDA para extração dos topiocs mais relevantes a partir da coluna criada tf-idf
N_TOKENS = 9

corpus = np.array(news_2016.tfidf.tolist())
lda_model = LDA(n_components=N_TOKENS, max_iter=100, random_state=SEED)
lda_model.fit(corpus)

## Visualize os tópicos usando a ferramenta pyLDAVis


In [ ]:
#pyLDavis nao foi possivel executar

# Atribua a cada text, um (e apenas um) tópic.


Crie uma coluna topic onde o valor é exatamente o tópico que melhor caracteriza o documento de acordo com o algoritmo de LDA.

In [ ]:
def get_topic(tfidf: np.array):

    tfidfarray = np.matrix([tfidf])
    array = np.asarray(tfidfarray)
    topics = LDA.transform(array)

    return np.argmax(topics)

news_2016['topic'] = news_2016.tfidf.progress_map(get_topic)

# Número de documentos vs tópicos

Esse gráfico nos mostra quantos documentos foram caracterizados por cada tópico.

In [ ]:
with sns.axes_style("ticks"):
    sns.set_context("talk")
    ax = news_2016['topic'].value_counts().sort_values().plot(kind = 'barh')
    ax.yaxis.grid(True)
    ax.set_ylabel("Tópico")
    ax.set_xlabel("Número de notícias (log)")
    sns.despine(offset = 10)
    ax.set_xscale("log")

# Crie uma nuvem de palavra para cada tópico.

Use as colunas spacy_lemma e topic para essa tarefa.

In [ ]:
def plot_wordcloud(text:str, ax:plt.Axes) -> plt.Axes:
    #Ficou por fazer.

    
def plot_wordcloud_for_a_topic(topic:int, ax:plt.Axes) -> plt.Axes:
    topic_news = news_2016[news_2016['topic'] == topic]
    list_of_words = chain(*topic_news.spacy_lemma.values.tolist())
    string_complete = ' '.join(list_of_words)
    if not string_complete:
        return None
    return plot_wordcloud(string_complete, ax)

fig, axis = plt.subplots(3, 3, figsize=(16, 12))

axis_  = axis.flatten()
for idx, ax in enumerate(axis_):
    ax_ = plot_wordcloud_for_a_topic(idx + 1, ax)
    if ax_ is None:
        plt.delaxes(ax)
        continue
    ax.set_title(f"Tópico {idx + 1}")
fig.tight_layout()

# Crie uma nuvem de entidades para cada tópico.

Use as colunas spacy_lemma e topic para essa tarefa.

In [ ]:
def plot_wordcloud_entities_for_a_topic(topic:int, ax:plt.Axes) -> plt.Axes:
    topic_news = news_2016[news_2016['topic'] == topic]
    list_of_docs = topic_news.spacy_ner.apply(lambda l : [w.replace(" ", "_") for w in l])
    list_of_words = chain(*list_of_docs)
    string_complete = ' '.join(list_of_words)
    if not len(string_complete):
        return None
    return plot_wordcloud(string_complete, ax)

fig, axis = plt.subplots(3, 3, figsize=(16, 12))

axis_  = axis.flatten()
for idx, ax in enumerate(axis_):
    ax_ = plot_wordcloud_entities_for_a_topic(idx + 1, ax)
    if ax_ is None:
        plt.delaxes(ax)
        continue
    ax.set_title(f"Tópico {idx + 1}")
fig.tight_layout()